In [1]:
%load_ext sql

In [2]:
%sql ibm_db_sa://xvf28527:dpt%5Ewh8wd36hbz54@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB

'Connected: xvf28527@BLUDB'

### Question 1
## --Write and execute a SQL query to list the school names, community names, and average attendance for communities with a hardship index of 98.

In [3]:
%%sql 


select sc.COMMUNITY_AREA_NAME,sc.NAME_OF_SCHOOL,sc.AVERAGE_STUDENT_ATTENDANCE
from CHICAGO_PUBLIC_SCHOOLS as sc 
left join CHICAGO_SOCIOECONOMIC_DATA as sd
on sd.COMMUNITY_AREA_NUMBER= sc.COMMUNITY_AREA_NUMBER
where sd.HARDSHIP_INDEX = 98;


 * ibm_db_sa://xvf28527:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


community_area_name,name_of_school,average_student_attendance
RIVERDALE,George Washington Carver Military Academy High School,91.60%
RIVERDALE,George Washington Carver Primary School,90.90%
RIVERDALE,Ira F Aldridge Elementary School,92.90%
RIVERDALE,William E B Dubois Elementary School,93.30%


### Question 2
### Write and execute a SQL query to list all crimes that took place at a school. Include case number, crime type and community name.

In [4]:
%%sql 
select cr.case_number, cr.primary_type,cd.COMMUNITY_AREA_NAME
from chicago_crime_data as cr
right join CHICAGO_SOCIOECONOMIC_DATA as cd
on cr.COMMUNITY_AREA_NUMBER= cd.COMMUNITY_AREA_NUMBER
where cr.LOCATION_DESCRIPTION like 'SCHOOL%';

 * ibm_db_sa://xvf28527:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


case_number,primary_type,community_area_name
HK577020,NARCOTICS,Rogers Park
HL725506,BATTERY,Lincoln Square
HH639427,BATTERY,Austin
HS200939,CRIMINAL DAMAGE,Austin
HT315369,ASSAULT,East Garfield Park
HP716225,BATTERY,Douglas
HL353697,BATTERY,South Shore
HS305355,NARCOTICS,Brighton Park
JA460432,BATTERY,Ashburn
HR585012,CRIMINAL TRESPASS,Ashburn


# Exercise 2: Creating a View

For privacy reasons, you have been asked to create a view that enables users to select just the school name and the icon fields from the CHICAGO_PUBLIC_SCHOOLS table. By providing a view, you can ensure that users cannot see the actual scores given to a school, just the icon associated with their score. You should define new names for the view columns to obscure the use of scores and icons in the original table.

## Question 1
Write and execute a SQL statement to create a view showing the columns listed in the following table, with new column names as shown in the second column

In [10]:
%%sql 
drop view chicago_public;
create view chicago_public as
select "NAME_OF_SCHOOL" as school_name,"Safety_Icon" as afety_rating,
"Family_Involvement_Icon" as family_rating,
"Environment_Icon" as environment_rating,
"Instruction_Icon" as instruction_rating,
"Leaders_Icon" as leaders_rating,
"Teachers_Icon" as teacher_rating
 from CHICAGO_PUBLIC_SCHOOLS;

select * from chicago_public limit 5;

 * ibm_db_sa://xvf28527:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.
Done.
Done.


school_name,afety_rating,family_rating,environment_rating,instruction_rating,leaders_rating,teacher_rating
Abraham Lincoln Elementary School,Very Strong,Very Strong,Strong,Strong,Weak,Strong
Adam Clayton Powell Paideia Community Academy Elementary School,Average,Strong,Strong,Very Strong,Weak,Strong
Adlai E Stevenson Elementary School,Strong,NDA,Average,Weak,Weak,NDA
Agustin Lara Elementary Academy,Average,Average,Average,Weak,Weak,Average
Air Force Academy High School,Average,Strong,Strong,Average,Weak,Average


1.Write and execute a SQL statement that returns all of the columns from the view.

In [12]:
%sql select * from chicago_public limit 3;

 * ibm_db_sa://xvf28527:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


school_name,afety_rating,family_rating,environment_rating,instruction_rating,leaders_rating,teacher_rating
Abraham Lincoln Elementary School,Very Strong,Very Strong,Strong,Strong,Weak,Strong
Adam Clayton Powell Paideia Community Academy Elementary School,Average,Strong,Strong,Very Strong,Weak,Strong
Adlai E Stevenson Elementary School,Strong,NDA,Average,Weak,Weak,NDA




Write and execute a SQL statement that returns just the school name and leaders rating from the view.

In [18]:
%sql select school_name,leaders_rating from chicago_public limit 2;

 * ibm_db_sa://xvf28527:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
Done.


school_name,leaders_rating
Abraham Lincoln Elementary School,Weak
Adam Clayton Powell Paideia Community Academy Elementary School,Weak


# Exercise 3: Creating a Stored Procedure

The icon fields are calculated based on the value in the corresponding score field. You need to make sure that when a score field is updated, the icon field is updated too. To do this, you will write a stored procedure that receives the school id and a leaders score as input parameters, calculates the icon setting and updates the fields appropriately.

## Question 1
Write the structure of a query to create or replace a stored procedure called UPDATE_LEADERS_SCORE that takes a in_School_ID parameter as an integer and a in_Leader_Score parameter as an integer. Don't forget to use the #SET TERMINATOR statement to use the @ for the CREATE statement terminator.

In [27]:
%%sql
--#SET TERMINATOR @
CREATE PROCEDURE UPDATE_LEADERS_SCORE(IN in_School_ID INTEGER,in_Leader_Score INTEGER)
LANGUAGE SQL 
MODIFIES SQL DATA 
BEGIN 

	IF in_Leader_Score < 20 THEN                       
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET "Leaders_Icon" = 'VERY_WEAK';
     
    ELSEIF in_Leader_Score < 40 THEN                       
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET "Leaders_Icon" = 'WEAK';
        
    ELSEIF in_Leader_Score < 60 THEN                       
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET  "Leaders_Icon" = 'AVG';
        
    ELSEIF in_Leader_Score < 80 THEN                       
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET "Leaders_Icon" = 'STRONG';

	 ELSEIF in_Leader_Score < 100 THEN                      
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET "Leaders_Icon" = 'VERY_STRONG';
        
    ELSE
        ROLLBACK WORK;

    END IF;       
    
    ELSE:
    COMMIT WORK;

END 
@

 * ibm_db_sa://xvf28527:***@dashdb-txn-sbox-yp-lon02-13.services.eu-gb.bluemix.net:50000/BLUDB
(ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: Statement Execute Failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0778N  End label "@" is not the same as the begin label.  LINE NUMBER=35.  SQLSTATE=428D5 SQLCODE=-778
[SQL: --#SET TERMINATOR @
CREATE PROCEDURE UPDATE_LEADERS_SCORE(IN in_School_ID INTEGER,in_Leader_Score INTEGER)
LANGUAGE SQL 
MODIFIES SQL DATA 
BEGIN 

	IF in_Leader_Score < 20 THEN                       
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET "Leaders_Icon" = 'VERY_WEAK';
     
    ELSEIF in_Leader_Score < 40 THEN                       
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET "Leaders_Icon" = 'WEAK';
        
    ELSEIF in_Leader_Score < 60 THEN                       
        UPDATE CHICAGO_PUBLIC_SCHOOLS
        SET  "Leaders_Icon" = 'AVG';
        
    ELSEIF in_Leader_Score < 80 THEN                       
        UPDATE CHICAGO_PUBLIC_SCHOOLS
